In [1]:
import pandas as pd
import numpy as np
import json
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    libgcc-ng-9.3.0            |      h5dbcf3e_17         7.8 MB  conda-forge
    openssl-1.1.1i             |       h7f98852_0         2.1 MB  conda-forge
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    libgomp-9.3.0              |      h5dbcf3e_17         378 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    _openm

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [4]:
sydney_lat = -33.8688
syndey_long = 151.2093

melb_lat = -37.8136
melb_long = 144.9631

bne_lat = -27.4698
bne_long = 153.0251

<h2>Sydney Dataset</h2>

In [5]:
search_query = '100'
radius = 3000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, sydney_lat, syndey_long, radius, LIMIT)
venues = requests.get(url).json()

In [6]:
venues = requests.get(url).json()["response"]['groups'][0]['items']
    
sydney_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
sydney_venues =sydney_venues.loc[:, filtered_columns]

# filter the category for each row
sydney_venues['venue.categories'] = sydney_venues.apply(get_category_type, axis=1)

# clean columns
sydney_venues.columns = [col.split(".")[-1] for col in sydney_venues.columns]

sydney_venues['City'] = 'Sydney'
sydney_venues.head()

,name,categories,lat,lng,City
0,UNIQLO,Clothing Store,-33.869744,151.208319,Sydney
1,Gumption by Coffee Alchemy,Coffee Shop,-33.869440,151.207700,Sydney
2,The Strand Arcade,Shopping Mall,-33.869420,151.207630,Sydney
3,State Theatre,Theater,-33.871291,151.207049,Sydney
4,Haigh's Chocolates,Candy Store,-33.869207,151.207129,Sydney


<h2>Melbourne Dataset</h2>

In [7]:
search_query = '100'
radius = 3000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, melb_lat, melb_long, radius, LIMIT)
venues = requests.get(url).json()

In [8]:
venues = requests.get(url).json()["response"]['groups'][0]['items']
    
melb_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
melb_venues =melb_venues.loc[:, filtered_columns]

# filter the category for each row
melb_venues['venue.categories'] = melb_venues.apply(get_category_type, axis=1)

# clean columns
melb_venues.columns = [col.split(".")[-1] for col in melb_venues.columns]

melb_venues['City'] = 'Melbourne'
melb_venues.head()

,name,categories,lat,lng,City
0,Tipo 00,Italian Restaurant,-37.813527,144.961978,Melbourne
1,Kirk's Wine Bar,Wine Bar,-37.813661,144.961351,Melbourne
2,Brother Baba Budan,Coffee Shop,-37.813445,144.962137,Melbourne
3,Little Rogue Coffee,Coffee Shop,-37.810986,144.964059,Melbourne
4,B'cos Brazil,Brazilian Restaurant,-37.815486,144.963085,Melbourne


<h2>Brisbane Dataset</h2>

In [9]:
search_query = '100'
radius = 3000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, bne_lat, bne_long, radius, LIMIT)
venues = requests.get(url).json()

In [10]:
venues = requests.get(url).json()["response"]['groups'][0]['items']
    
bne_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
bne_venues =bne_venues.loc[:, filtered_columns]

# filter the category for each row
bne_venues['venue.categories'] = bne_venues.apply(get_category_type, axis=1)

# clean columns
bne_venues.columns = [col.split(".")[-1] for col in bne_venues.columns]

bne_venues['City'] = 'Brisbane'
bne_venues.head()

,name,categories,lat,lng,City
0,John Mills Himself,Coffee Shop,-27.471452,153.025440,Brisbane
1,The Dark Chocolatier by the Noosa Chocolate Fa...,Chocolate Shop,-27.467830,153.025723,Brisbane
2,Maru Korean Restaurant,Korean Restaurant,-27.470260,153.026710,Brisbane
3,Cartel Coffee,Coffee Shop,-27.468448,153.024845,Brisbane
4,W Brisbane,Hotel,-27.470120,153.021970,Brisbane


In [11]:
all_venues = sydney_venues
all_venues = all_venues.append(melb_venues)
all_venues = all_venues.append(bne_venues)
all_venues.head()

,name,categories,lat,lng,City
0,UNIQLO,Clothing Store,-33.869744,151.208319,Sydney
1,Gumption by Coffee Alchemy,Coffee Shop,-33.869440,151.207700,Sydney
2,The Strand Arcade,Shopping Mall,-33.869420,151.207630,Sydney
3,State Theatre,Theater,-33.871291,151.207049,Sydney
4,Haigh's Chocolates,Candy Store,-33.869207,151.207129,Sydney


In [12]:
all_onehot = pd.get_dummies(all_venues[['categories']], prefix="", prefix_sep="")
all_onehot['City'] = all_venues['City']

fixed_columns = [all_onehot.columns[-1]] + list(all_onehot.columns[:-1])
all_onehot = all_onehot[fixed_columns]

all_onehot.head()

,City,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bar,...,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Tour Provider,Trail,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar
0,Sydney,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Sydney,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Sydney,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Sydney,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,Sydney,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
all_grouped = all_onehot.groupby('City').mean().reset_index()
all_grouped

num_top_venues = 10

for city in all_grouped['City']:
    print("----"+city+"----")
    temp = all_grouped[all_grouped['City'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brisbane----
                 venue  freq
0                 Café  0.11
1                Hotel  0.07
2                 Park  0.06
3                  Bar  0.06
4             Beer Bar  0.05
5          Coffee Shop  0.03
6  Japanese Restaurant  0.03
7               Bakery  0.02
8         Burger Joint  0.02
9     Pedestrian Plaza  0.02


----Melbourne----
               venue  freq
0       Cocktail Bar  0.09
1        Coffee Shop  0.07
2                Bar  0.07
3   Asian Restaurant  0.04
4               Café  0.04
5       Dessert Shop  0.04
6  Korean Restaurant  0.03
7              Plaza  0.03
8           Wine Bar  0.02
9          Bookstore  0.02


----Sydney----
                   venue  freq
0                   Café  0.10
1           Cocktail Bar  0.05
2                  Hotel  0.05
3        Thai Restaurant  0.05
4             Restaurant  0.04
5            Coffee Shop  0.04
6                   Park  0.04
7                Theater  0.04
8     Chinese Restaurant  0.03
9  Australian Restau

In [14]:
kclusters = 3

all_clustering = all_grouped.drop("City",1)

kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(all_clustering)

kmeans.labels_[0:10]

array([0, 1, 2], dtype=int32)

In [15]:
all_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
all_grouped.head()

,Cluster Labels,City,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,...,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Tour Provider,Trail,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar
0,0,Brisbane,0.00,0.02,0.00,0.02,0.01,0.01,0.00,0.02,...,0.00,0.00,0.02,0.02,0.01,0.00,0.01,0.01,0.01,0.00
1,1,Melbourne,0.01,0.01,0.00,0.04,0.00,0.01,0.01,0.01,...,0.00,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.02,0.02
2,2,Sydney,0.00,0.01,0.01,0.00,0.00,0.03,0.00,0.02,...,0.01,0.00,0.05,0.04,0.00,0.01,0.01,0.00,0.02,0.00


In [16]:
from sklearn.metrics.pairwise import euclidean_distances

#Brisbane to Melbourne
dists = euclidean_distances(kmeans.cluster_centers_[[0,1]])
print("Brisbane to Melbourne\n", dists,"\n")

#Brisbane to Sydney
dists = euclidean_distances(kmeans.cluster_centers_[[0,2]])
print("Brisbane to Sydney\n", dists,"\n")
                                                    
#Melbourne to Sydney
dists = euclidean_distances(kmeans.cluster_centers_[[1,2]])
print("Melbourne to Sydney\n",dists,"\n")

Brisbane to Melbourne
 [[0.         0.17944358]
 [0.17944358 0.        ]] 

Brisbane to Sydney
 [[0.         0.12727922]
 [0.12727922 0.        ]] 

Melbourne to Sydney
 [[0.         0.16673332]
 [0.16673332 0.        ]] 

